## Character Level ConvNets project

## Load Datasets


In [8]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pandas as pd

In [17]:
df = pd.read_csv('./yahoo_answers_csv/train.csv', header = None, nrows = 10)
Y_train = df[0]
df.drop(df.columns[0],axis=1,inplace=True)
df[df.columns[0]] = df[df.columns[0]].map(str) + df[df.columns[1]].map(str)+ df[df.columns[2]].map(str)
df.drop(df.columns[[1,2]],axis=1,inplace=True)
l = 2
X_train = df[df.columns[0]].apply(lambda x: x[:l] )
print(X_train)


0    wh
1    Wh
2    Wh
3    Ho
4    In
5    Wh
6    Wh
7    Is
8    Wh
9    Ho
Name: 1, dtype: object


In [21]:
def oneHotEncoder(x):
    idxs = np.fromstring(x,dtype=np.uint8)-97
    one_hot = tf.one_hot(idxs,70, dtype=tf.uint8)
    sess = tf.InteractiveSession()
    return one_hot.eval()

oneHotEncoder("hello")
X = X_train.apply(lambda x: oneHotEncoder(x))
print(X)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app


0    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
4    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
5    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
6    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
7    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
8    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
9    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: 1, dtype: object


In [ ]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
L = 1024 # 1014, 1024 for large 256 for small
NOUT = 2048 # 2048 for large, 1024 for small 
NUM_CHARS = 70
STD_INIT = 0.02 # 0.02 for large, 0.05 for small 
X = tf.placeholder(tf.float32, [None, NUM_CHARS, L])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def simple_model(X,y):    
    conv1 = tf.contrib.layers.conv1d(X, filters=1, kernel_size=7, stride=1, padding='SAME', 
                                   kernel_initializer=tf.initializers.random_normal(mean=0, stddev=STD_INIT), 
                                     # is this correct
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv1") 
    pool1 = tf.nn.max_pool(value=conv1, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool1") 
    
    conv2 = tf.contrib.layers.conv1d(pool1, filters=1, kernel_size=7, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv2") 
    pool2 = tf.nn.max_pool(value=conv2, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool2")
    
    conv3 = tf.contrib.layers.conv1d(pool2, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv3") 
    
    conv4 = tf.contrib.layers.conv1d(conv3, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv4") 
    
    conv5 = tf.contrib.layers.conv1d(conv4, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv5") 
    
    conv6 = tf.contrib.layers.conv1d(conv5, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv6") 
    pool6 = tf.nn.max_pool(value=conv6, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool6")
    
    fc7 = tf.contrib.layers.fully_connected(inputs=tf.contrib.layers.flatten(pool6), num_outputs=NOUT, reuse=None, 
                                      scope="fc7")
    fc8 = tf.contrib.layers.fully_connected(inputs=tf.contrib.layers.flatten(fc7), num_outputs=NOUT, reuse=None, 
                                      scope="fc8")
    
    return fc8

y_out = simple_model(X,y)

# define our loss
total_loss = tf.losses.hinge_loss(tf.one_hot(y,10),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)